<a href="https://colab.research.google.com/github/sheikhi-a/Animated-Visualization/blob/main/Volleyball_game_analysis_using_Image_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install opencv-python mediapipe numpy scikit-learn

import os
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.1/36.1 MB 16.3 MB/s eta 0:00:00


In [3]:

print(os.listdir('/content'))  # List all files in content directory
video_path = '/content/volmatch2.mp4'  # Update with your file name


def extract_frames(video_path, output_dir='/content/frames'):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    cap = cv2.VideoCapture(video_path)
    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_path = os.path.join(output_dir, f"frame_{frame_count:04d}.jpg")
        cv2.imwrite(frame_path, frame)
        frame_count += 1

    cap.release()
    print(f"Extracted {frame_count} frames to {output_dir}")

extract_frames(video_path)



mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

def detect_keypoints_from_frames(frame_dir, output_csv='/content/keypoints.csv'):
    data = []
    for frame_file in sorted(os.listdir(frame_dir)):
        frame_path = os.path.join(frame_dir, frame_file)
        frame = cv2.imread(frame_path)

        # Convert to RGB and detect pose landmarks
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(rgb_frame)

        if results.pose_landmarks:
            keypoints = [(lm.x, lm.y) for lm in results.pose_landmarks.landmark]
            flattened_keypoints = [coord for point in keypoints for coord in point]  # Flatten list
            data.append([frame_file] + flattened_keypoints)

    # Save keypoints to CSV
    num_keypoints = len(data[0]) - 1  # Exclude frame column
    columns = ['Frame'] + [f'Keypoint_{i}' for i in range(num_keypoints)]
    df = pd.DataFrame(data, columns=columns)
    df.to_csv(output_csv, index=False)
    print(f"Keypoints saved to {output_csv}")

detect_keypoints_from_frames('/content/frames')

['.config', 'volmatch2.mp4', 'frames', 'sample_data']
Extracted 363 frames to /content/frames
Keypoints saved to /content/keypoints.csv


In [6]:
def get_frame_at_second(video_path, specific_second):
    cap = cv2.VideoCapture(video_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_number = specific_second * fps
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
    ret, frame = cap.read()
    cap.release()
    return frame if ret else None

mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

def detect_keypoints(frame):
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb_frame)
    if results.pose_landmarks:
        keypoints = [(lm.x, lm.y) for lm in results.pose_landmarks.landmark]
        return keypoints
    return None
EXPECTED_NUM_KEYPOINTS = 33

def compute_features(keypoints):
    if not keypoints or len(keypoints) != EXPECTED_NUM_KEYPOINTS:
        keypoints = keypoints[:EXPECTED_NUM_KEYPOINTS] + [(0, 0)] * (EXPECTED_NUM_KEYPOINTS - len(keypoints))
    features = []
    for i in range(EXPECTED_NUM_KEYPOINTS - 1):
        for j in range(i + 1, EXPECTED_NUM_KEYPOINTS):
            distance = np.linalg.norm(np.array(keypoints[i]) - np.array(keypoints[j]))
            features.append(distance)
    return features[:528]

np.random.seed(42)
dummy_data = np.random.rand(100, 528)
dummy_labels = np.random.choice(["Zone 2", "Zone 4"], 100)

X_train, X_test, y_train, y_test = train_test_split(dummy_data, dummy_labels, test_size=0.2, random_state=42)

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

def predict_next_spike_zone(video_path, specific_second):
    frame = get_frame_at_second(video_path, specific_second)
    if frame is None:
        return "Could not extract frame at the specified second."
    keypoints = detect_keypoints(frame)
    if not keypoints:
        return "Could not detect keypoints in the frame."
    features = compute_features(keypoints)
    if not features:
        return "Features could not be computed from keypoints."
    prediction = model.predict([features])[0]
    return f"Predicted Zone for Next Spike: {prediction}"

video_path = "/content/volmatch2.mp4" # this is my video
specific_second = 13 # this is my specific time

result = predict_next_spike_zone(video_path, specific_second)
print(result)


Predicted Zone for Next Spike: Zone 4
